In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import seaborn as sns
import uncertainties as uc
from uncertainties.umath import sqrt
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.signal import correlate
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.fft import fft
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

In [16]:
plt.style.use('Solarize_Light2')   #Options
                                    #Solarize_Light2
                                    #_classic_test_patch
                                    #_mpl-gallery
                                    #_mpl-gallery-nogrid
                                    #bmh
                                    #classic
                                    #dark_background
                                    #fast
                                    #fivethirtyeight
                                    #ggplot
                                    #grayscale
                                    #seaborn-v0_8
                                    #seaborn-v0_8-bright
                                    #seaborn-v0_8-colorblind
                                    #seaborn-v0_8-dark
                                    #seaborn-v0_8-dark-palette
                                    #seaborn-v0_8-darkgrid
                                    #seaborn-v0_8-deep
                                    #seaborn-v0_8-muted
                                    #seaborn-v0_8-notebook
                                    #seaborn-v0_8-paper
                                    #seaborn-v0_8-pastel
                                    #seaborn-v0_8-poster
                                    #seaborn-v0_8-talk
                                    #seaborn-v0_8-ticks
                                    #seaborn-v0_8-white
                                    #seaborn-v0_8-whitegrid
                                    #tableau-colorblind10

plt.rcParams.update({

    'figure.dpi': 225,                 # Resolution of figures
    'font.size': 10,                   # Default font size
    'font.family': 'sans-serif',       # Font family
    'font.sans-serif': ['DejaVu Sans'],
    
    'axes.titlesize': 10,              # Title font size
    'axes.labelsize': 8,              # Axis label font size
    'axes.linewidth': 0.8,             # Width of the axis lines
    'axes.grid': True,                 # Show grid
    # 'grid.color': '#dddddd',
    'grid.linestyle': '--',            # Grid line style (dashed)
    'grid.linewidth': 0.5,             # Grid line width
    
    'xtick.labelsize': 8,             # X-axis tick label size
    'ytick.labelsize': 8,             # Y-axis tick label size
    'xtick.direction': 'in',           # X-axis tick direction
    'ytick.direction': 'in',           # Y-axis tick direction

    'lines.linewidth': 0.8,            # Line width for plots
     'lines.markersize': 0.3,             # Marker size for scatter plots
     'lines.marker': 'o',               # Default marker shape

    'legend.frameon': True,            # Frame around legend
    'legend.framealpha': 0.8,          # Transparency of legend frame
    'legend.fontsize': 9,             # Font size in legend
    'legend.loc': 'best',              # Default legend location
})

In [17]:
# Errors functions
def rSquared(f_model, x, y, Params):
    y_pred = f_model(*Params, x)  
    ss_res = np.sum((y - y_pred) ** 2)  
    ss_tot = np.sum((y - np.mean(y)) ** 2)  
    return 1 - (ss_res / ss_tot)

def Residuals(y_measured,y_modelled):
    return y_measured - y_modelled

def MeanSquareError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def RootMeanSquareError(y_measured,y_modelled):
    return np.sqrt(MeanSquareError(y_measured,y_modelled))

def MeanAbsError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum(np.abs(y_measured-y_modelled))))

def ConfidenceInterval(y_measured,y_modelled,confInt=95):
    if(confInt == 95):
        Z = 1.96
    elif(confInt == 90):
        Z = 1.645
    elif(confInt == 99):
        Z = 2.576
    return Z*np.sqrt((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def ArrayWithStaticUnc(Array,unc):
    a = []
    b = []
    for i in range(len(Array)):
            a.append(uc.ufloat(Array[i],unc))
            b.append(unc)
    return np.array(a), np.array(b)

def UnpackUncArray(Array):
     value = [Array[i].nominal_value for i in range(len(Array))]
     std = [Array[i].std_dev for i in range(len(Array))]
     return value,std 

# Non Linear Curve Fiting Function
def NonLinAnalysis(f_model, x, y, ParamsArr, weights):
    if ParamsArr is None:
        Params = None
        std = None 
        R2 = rSquared(f_model,x,y,Params)
    else:
        Params, cov = curve_fit(f_model, x, y, p0=ParamsArr, sigma=weights)
        std = np.sqrt(np.diag(cov))
        R2 = rSquared(f_model,x,y,Params)
    return Params, std, R2
 
def RR2(y,y_pred):
   ss_res = np.sum((y - y_pred) ** 2)
   ss_tot = np.sum((y - np.mean(y)) ** 2)
   R2 = 1 - (ss_res / ss_tot)
   return R2

def sortByArray1(array1, array2):
    sorted_pairs = sorted(zip(array1, array2))
    sorted_array1, sorted_array2 = zip(*sorted_pairs)
    return list(sorted_array1), list(sorted_array2)

## Inputting data

In [18]:
Task1 = {"400mA":1,"480mA":1,"560mA":1,"640mA":1,"720mA":1,"800mA":1,"880mA":1,"960mA":1,"1040mA":1,"1120mA":1,"1200mA":1}
for key, _ in Task1.items():
    Task1[key] = pd.read_csv(f"Task1/{key}.csv")
    Task1[key].iloc[:,1:3] = Task1[key].iloc[:,1:3]*(np.pi/2)

## Basic functions

In [19]:
def angular_velocity_model(omega, M0, J, omega0, D):
    numerator = omega
    denominator = np.sqrt((J * (omega0**2 - omega**2))**2 + (D * omega)**2)
    return numerator / denominator

def amplitude_model(M0, J, omega0, delta, omega):
    m = M0
    n = J * np.sqrt((omega0**2 - omega**2)**2 + (2* delta * omega)**2)
    return m / n

def phase_shift_model(omega, omega0, delta):
    if omega < omega0:
        p = (2) *omega * delta
        r = omega0**2 - omega**2
        return - np.arctan(p/r)
    elif omega > omega0: 
        p = (2) *omega * delta
        r = omega**2 - omega0**2
    return - np.pi + np.arctan(p/r)

def resonance_omega(omega0, delta):
    return np.sqrt(omega0**2 - 2*delta**2)

def resonance_amplitude(A,omega0, J, D, delta):
    if delta < omega0/np.sqrt(2):
        d = A
        w = 2*np.sqrt(omega0**2 - 2*delta**2)*delta*D
        return d/w
    else:
        return A/D

def quality_factor(omega0, delta):
    a = omega0
    b = 2*delta
    return a/b

def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    return filtered_data


def polynomial_model(t, a, b, c):
    return a * t**2 + b * t + c

def sinusoidal_model(t, A, alpha, beta, C):
    return A * np.sin(alpha * t + beta)+ C

def cosinusoidal_model(t, A, alpha, beta, C):
    return A * np.cos(alpha * t + beta)+ C

def exponential_model(t, C, delta, beta):
    return C * np.exp(- delta* t) * np.cos(alpha*t - beta)

def driven_motion(t, M0, phi, delta, J, omega0):
    m = M0
    n = J * np.sqrt((omega0**2 - omega**2)**2 + (2* delta * omega)**2)
    A = m/n
    return A * np.sin(omega * t + phi) 

def damped_motion(t, omegad, C, delta, alpha):
    return C * np.exp(- delta * t)*np.cos(omegad*t-alpha)

def equation_of_motion(t, M0, omega, phi, C, delta, alpha, omegad, J, omega0):
    m = M0
    n = J * np.sqrt((omega0**2 - omega**2)**2 + (2* delta * omega)**2)
    A = m/n
    return [A * np.sin(omega * t + phi) + C * np.exp(- delta * t)*np.cos(omegad*t - alpha)]

def DampedOscillation(x,C,d,w,a):
    return C*np.exp(-d*x)*np.cos(w*x-a)

def find_zeros(y, min_distance=1):
    zero_crossings = np.where(np.diff(np.sign(y)))[0]
    filtered_zeros = []
    last_zero = -np.inf
    for zero in zero_crossings:
        if zero - last_zero >= min_distance:
            filtered_zeros.append(zero)
            last_zero = zero
    return np.array(filtered_zeros)

## Task I

In [ ]:
for i, _ in enumerate(Task1):
    AbsError = np.max([np.abs(np.max(Task1[_]["U2(V)"])),np.abs(np.min(Task1[_]["U2(V)"]))])
    print(f"Dataset: {_} Uncertainty: ±{AbsError} \n Adding Error...")
    UncertaintyColumnName = "U1(V)mitUnc"
    Task1[_][UncertaintyColumnName] = ArrayWithStaticUnc(Task1[_]["U1(V)"],AbsError)[0]
    print(f"Uncertainity added into DataFrame as {UncertaintyColumnName} \n") 

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(15, 10))
axes = axes.flatten()
fs = 5000
T1TrimLines = {"400mA":[2000/fs,15.1],"480mA":[2000/fs,9.99],"560mA":[2000/fs,8.5],
               "640mA":[2000/fs,7],"720mA":[0.75,7],"800mA":[1000/fs,4.6],
               "880mA":[2500/fs,4.9],"960mA":[1,5],"1040mA":[0.75,5],"1120mA":[1,3.3],"1200mA":[2000/fs,3]}
for i, _ in enumerate(Task1):
    axes[i].plot(Task1[_]["t (s)"],Task1[_]["U1(V)"],label="Measured")
    axes[i].fill_between(Task1[_]["t (s)"],Task1[_]["U1(V)"]-Task1[_]["U1(V)mitUnc"][0].std_dev,Task1[_]["U1(V)"]+Task1[_]["U1(V)mitUnc"][0].std_dev,
                         color="red",alpha=0.3,label="Measured Uncertainty")
    axes[i].vlines(T1TrimLines[_],ymin=np.min(Task1[_]["U1(V)"]),ymax=np.max(Task1[_]["U1(V)"]),color='gray',linestyle="--",label="Trim Lines")
    axes[i].set_title(f"{_}")
    axes[i].set_xlabel("Time (s)")
    axes[i].set_ylabel("Deflection Angle (rad)")
    axes[i].legend(loc="upper right")
    Task1[_] = Task1[_].iloc[int(T1TrimLines[_][0]*fs):int(T1TrimLines[_][1]*fs)].reset_index(drop=True)
for j in range(len(Task1), len(axes)):
    axes[j].axis('off')

plt.suptitle("No External Drive Results")
plt.tight_layout()
plt.show()

In [ ]:
def DampedOscillation(x,C,d,w,a):
    return C*np.exp(-d*x)*np.cos(w*x-a)

def FindingW0(x,C,d,w0,a):
    return C*np.exp(-d*x)*np.cos(np.sqrt(w0**2-d**2)*x-a)

ChosenGraphXY = []

dampers = []
Wdpers = []
WdpersUnc = []
W0pers = []
W0persUnc = []
fig, axes = plt.subplots(4, 3, figsize=(15, 10))
axes = axes.flatten()
for i,_ in enumerate(Task1):
    data = Task1[_].iloc[:,0:2].rolling(1000,center=True).mean().dropna().reset_index(drop=True)
    weights = np.ones(len(data))*np.asarray(Task1[_]["U1(V)mitUnc"][0].std_dev).flatten()
    Params, std, R2= NonLinAnalysis(DampedOscillation,data["t (s)"],data["U1(V)"],[1,0.01,1.5,0.75],weights=weights)
    a,b,c = NonLinAnalysis(FindingW0,data["t (s)"],data["U1(V)"],[1,0.01,1.5,0.75],weights)
    W0pers.append(a[2])
    W0persUnc.append(b[2])
    dampers.append(Params[1])
    Wdpers.append(Params[2])
    WdpersUnc.append(std[2])
    y = DampedOscillation(data["t (s)"],Params[0],Params[1],Params[2],Params[3])
    R2 = RR2(data["U1(V)"],y)
    table_data = [
        [r"$\omega_d$", f"{Params[2]:.3g}±{std[0]:.3g}"],
        [r"$\delta$", f"{Params[1]:.3g}±{std[1]:.3g}"],
        [r"$R^2$", f"{R2:.5g}"],
        ]
    axes[i].table(cellText=table_data,
                  colLabels=["","Modelled"],
                  loc='left',
                  cellLoc='left',
                  bbox=[0.65,0.60,0.35,0.35],
                  colWidths=[0.12,0.3]
                  ,fontsize=8,zorder=100)
    axes[i].set_title(f"{_}")
    axes[i].set_xlabel("Time (s)")
    axes[i].fill_between(Task1[_]["t (s)"],Task1[_]["U1(V)"]-Task1[_]["U1(V)mitUnc"][0].std_dev,Task1[_]["U1(V)"]+Task1[_]["U1(V)mitUnc"][0].std_dev,
                         color="red",alpha=0.3,label="Raw Uncertainty")
    axes[i].set_ylabel("Deflection Angle (rad)")
    axes[i].plot(data["t (s)"],data["U1(V)"],label="Filtered Measured")
    axes[i].plot(data["t (s)"],y,color="red",alpha=1,label="Modelled")
    axes[i].fill_between(data["t (s)"],DampedOscillation(data["t (s)"],Params[0]-std[0],Params[1]-std[1],Params[2]-std[2],Params[3]-std[2]),
                         DampedOscillation(data["t (s)"],Params[0]+std[0],Params[1]+std[1],Params[2]+std[2],Params[3]+std[2]),color="pink",alpha=1,label="Model Uncertainty")
    axes[i].legend(loc="lower right",fontsize=7)
for j in range(len(Task1), len(axes)):
    axes[j].axis('off')


def TwoArrayAverageMitUncertainty(values, uncertainties):
    values = np.array(values, dtype=float)
    weights = 1 / np.array(uncertainties,dtype=float)**2
    weighted_avg = np.sum(weights * np.array(values, dtype=float)) / np.sum(weights)
    uncertainty_avg = np.sqrt(1 / np.sum(weights))
    return weighted_avg, uncertainty_avg
W0 = uc.ufloat(TwoArrayAverageMitUncertainty(W0pers, W0persUnc)[0],TwoArrayAverageMitUncertainty(W0pers, W0persUnc)[1])
print(f"W0 Average is: {W0}")

plt.tight_layout()
plt.show()


In [ ]:
data = Task1["800mA"].iloc[:,0:2].rolling(1000,center=True).mean().dropna().reset_index(drop=True)
weights = np.ones(len(data))*np.asarray(Task1["800mA"]["U1(V)mitUnc"][0].std_dev).flatten()
Params, std, R2= NonLinAnalysis(DampedOscillation,data["t (s)"],data["U1(V)"],[1,0.01,1.5,0.75],weights)
y = DampedOscillation(data["t (s)"],Params[0],Params[1],Params[2],Params[3])
R2 = RR2(data["U1(V)"],y)
table_data = [
    [r"$\omega_d$", f"{Params[2]:.3g}±{std[0]:.3g}"],
    [r"$\delta$", f"{Params[1]:.3g}±{std[1]:.3g}"],
    [r"$R^2$", f"{R2:.5g}"],
    ]
plt.table(cellText=table_data,
              colLabels=["","Modelled"],
              loc='left',
              cellLoc='left',
              bbox=[0.65,0.60,0.35,0.35],
              colWidths=[0.12,0.3]
              ,fontsize=8,zorder=100)
plt.title("800mA")
plt.xlabel("Time (s)")
plt.ylabel("Deflection Angle (rad)")
# plt.plot(Task1["800mA"]["t (s)"],Task1["800mA"]["U1(V)"],label="Measured")
plt.fill_between(Task1["800mA"]["t (s)"],Task1["800mA"]["U1(V)"]-Task1["800mA"]["U1(V)mitUnc"][0].std_dev,Task1["800mA"]["U1(V)"]+Task1["800mA"]["U1(V)mitUnc"][0].std_dev,
                         color="red",alpha=0.3,label="Raw Uncertainty")
plt.plot(data["t (s)"],data["U1(V)"],label="Filtered Measured")
plt.plot(data["t (s)"],y,color="red",alpha=1,label="Modelled")
plt.fill_between(data["t (s)"],DampedOscillation(data["t (s)"],Params[0]-std[0],Params[1]-std[1],Params[2]-std[2],Params[3]-std[2]),
                     DampedOscillation(data["t (s)"],Params[0]+std[0],Params[1]+std[1],Params[2]+std[2],Params[3]+std[2]),color="pink",alpha=1,label="Model Uncertainty")
plt.legend(loc="lower right",fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
dampers1,Wdpers1 = np.array(dampers)**2,np.array(Wdpers)**2
dampers1,Wdpers1 = sortByArray1(dampers1,Wdpers1)
def Linear(x,slope,intercept):
    return np.array(x) * slope + intercept
WdParams,WdStd,WdR2= NonLinAnalysis(Linear,dampers1,Wdpers1,[-1,4],None)
WdR2 = RR2(Wdpers1,Linear(dampers1,WdParams[0],WdParams[1]))
RRR2 = RR2((WdParams[1]-dampers1),Linear(dampers1,WdParams[0],WdParams[1]))
WW0 = sqrt(uc.ufloat(WdParams[1],WdStd[1]))
table_data1 = [
    [r"$\omega_0$", f"{WW0.nominal_value:.5g}±{WW0.std_dev:.3g}",f"{WW0.nominal_value:.5g}±{WW0.std_dev:.3g}"],
    [r"Slope", f"{WdParams[0]:.5g},±{WdStd[1]:.3g}","-1"],
    [r"$R^2$", f"{WdR2:.5g}",""],
    [r"Comp.", r"$R^2$",f"{RRR2:.5g}"]
    ]
print(WdStd)
plt.subplots(1, figsize=(8, 4))
plt.table(cellText=table_data1,
        colLabels=["","Fitted","Comp."],
        loc='left',
        cellLoc='left',
        bbox=[0.70,0.70,0.25,0.25],
        colWidths=[0.12,0.27,0.27],
        fontsize=12,zorder=100)
plt.scatter(dampers1,Wdpers1,color="olive",s=10,edgecolors="k",linewidths=0.5)
plt.plot(dampers1,Linear(dampers1,WdParams[0],WdParams[1]),label="Fitted")
plt.fill_between(dampers1,Linear(dampers1,WdParams[0]-WdStd[0],WdParams[1]-WdStd[1]),Linear(dampers1,WdParams[0]+WdStd[0],WdParams[1]+WdStd[1]),alpha=0.3)
plt.plot(dampers1,(WdParams[1]-dampers1),label="Theo.")
plt.fill_between(dampers1,((WdParams[1]-WdStd[1])-dampers1),((WdParams[1]+WdStd[1])-dampers1),alpha=0.3)
plt.title(r"$\omega_d^{2}$ against $\delta^{2}$")
plt.ylabel(r"$\omega_d^{2}$ $rad/s$")
plt.xlabel(r"$\delta^{2}$ $rad/s$")
plt.legend(loc="lower left")
plt.show()

## Task II

In [ ]:
data = {}
t_fake = {}
totaldeflectionangle = {}
drivendeflection = {}
params = {}
omegafake = []
amplitudes = {}
angle_fit1 = {}
deflection_fit1 = {}

for i in range(0, 1):
    file_path = f"Task2/T2{i * 10 + 35}dV.csv"
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)  
    df["t (s)"] = pd.to_numeric(df["t (s)"].str.replace(",", "."), errors='coerce')
    df["U1(V)"] = pd.to_numeric(df["U1(V)"].str.replace(",", "."), errors='coerce')
    df["U2(V)"] = pd.to_numeric(df["U2(V)"].str.replace(",", "."), errors='coerce')
    df.dropna(inplace=True)
    remove_outliers(df, "U1(V)")
    remove_outliers(df, "U2(V)")
    U1 = df["U1(V)"].values
    U2 = df["U2(V)"].values
    time = df["t (s)"].values
    mask_U1 = np.diff(U1, prepend=np.nan) != 0
    t1 = time[mask_U1]
    u1 = U1[mask_U1] 
    mask_U2 = np.diff(U2, prepend=np.nan) != 0
    t2 = time[mask_U2]
    u2 = U2[mask_U2]
    t_common = np.sort(np.unique(np.concatenate((t1, t2)))) 
    u1_interp = ArrayWithStaticUnc(np.interp(t_common, t1, u1),16.5*10**(-3))[0]
    u2_interp = ArrayWithStaticUnc(np.interp(t_common, t2, u2),16.5*10**(-3))[0]
    #timephase = t_common - t2
    # Instead of accessing `nominal_value`, unpack into separate values and std deviations
    u1_values, _ = UnpackUncArray(u1_interp)
    u2_values, _ = UnpackUncArray(u2_interp)

    totaldeflectionangle[i] = np.array(u2_values) * 90 * np.pi / 180
    drivendeflection[i] = np.array(u1_values) * 90 * np.pi / 180 
    angle = totaldeflectionangle[i] - drivendeflection[i]
    initial_guess = [10, 1, 0,0]
    weights = None  # Assuming weights are not provided for now
    params, _ = curve_fit(sinusoidal_model, t_common, totaldeflectionangle[i], p0=initial_guess)
    A, alpha, beta, C = params
    angle_fit = sinusoidal_model(t_common, A, alpha, beta, C)
    R2 = RR2(angle_fit, totaldeflectionangle[i])
    print("Sinusoidal Fit: ", R2)

    angle_fit1[i] = angle_fit 
    initial_guess = [10, 1, 1,0]
    paramsd, _ = curve_fit(sinusoidal_model, t_common, drivendeflection[i], p0=initial_guess)
    Ad, alphad, betad, Cd = paramsd
    deflection_fit = sinusoidal_model(t_common, Ad, alphad, betad, Cd)
    R2 = RR2(deflection_fit, drivendeflection[i])
    print("Sinusoidal Fit: ", R2)

    deflection_fit1[i] = deflection_fit
    plt.figure(figsize=(8, 5))
    plt.plot(t_common, angle_fit, color="red")
    plt.plot(t_common, totaldeflectionangle[i], label="Driven Deflection", color="purple", zorder=1)
    plt.plot(t_common, drivendeflection[i], label="Damping Deflection", color="green", zorder=1)
    plt.plot(t_common, angle, label="Total Deflection", color="orange", zorder=2)
    plt.plot(t_common, deflection_fit, color="red", label="Sinusoidal Fits", zorder=2)
    plt.title(f"Total Deflection for {(i * 10 + 35) / 10} V Driving Voltage")
    plt.xlabel("Time (s)")
    plt.ylabel("Deflection Angle (rad)")
    plt.legend()
    plt.grid(True)
    plt.show()

    if i == 0:
        omega = A * alpha * np.cos(alpha * t_common + beta)  # rad/s
        omegad = Ad * alphad * np.cos(alphad * t_common + betad)
        omegatotal = omega - omegad 
        #theta = omega * timephase
        amplitudes = {"times": t_common, "values": omegatotal}
        plt.plot(t_common, omegatotal, label="Total Angle", color="blue")
        plt.title("Angular velocity vs. Time")
        plt.xlabel("Time (s)")
        plt.ylabel("Angular velocity (rad/s)")
        plt.show()

    t_fake[i] = t_common
    print(f"Parameters for {(i * 10 + 35) / 10} V: A = {A:.3f}, alpha = {alpha:.3f}, C = {C:.3f} and beta = {beta:.3f}")
    print(f"Parameters for {(i * 10 + 35) / 10} V: Ad = {Ad:.3f}, alphad = {alphad:.3f}, Cd = {Cd:.3f} and betad = {betad:.3f}")

In [ ]:
plt.plot(t_fake[0], omegatotal, label="Deflection Angle", color="blue")
plt.plot(t_fake[0], omegad, label="Damping Angle", color="green")
plt.plot(t_fake[0], omega, label="Driving Angle", color="red")
plt.title("Angular frequencies Using Best Fit Functions")
plt.xlabel("Time (s)")
plt.ylabel("Angular frequencies (rad/s)")
plt.legend()
plt.grid(True)
plt.show()

peaks_drive, _ = find_peaks(omega)
t_peaks_drive = t_fake[0][peaks_drive]

peaks_total, _ = find_peaks(omegatotal)
t_peaks_total = t_fake[0][peaks_total]

t_peak_drive = t_peaks_drive[0]
t_peak_total = t_peaks_total[0]

delta_t = t_peak_total - t_peak_drive

phase_shift = delta_t * omega
shifted_omegatotal = np.interp(t_fake[0], t_fake[0] - delta_t, omegatotal)
plt.figure(figsize=(10, 6))
plt.plot(t_fake[0], shifted_omegatotal, label="Shifted Resulting Velocity", color="purple", linestyle='--') 
plt.plot(t_fake[0], omega, label="Driving Velocity", color="blue")
plt.plot(t_fake[0], omegatotal, label="Resulting Velocity", color="green")
plt.title("Phase Shift and Resulting Velocity")
plt.xlabel("Time (s)")
plt.ylabel("Frequencies (rad/s)")
plt.legend()
plt.grid(True)
plt.show()
plt.plot(t_fake[0], phase_shift, label="Phase Shift", color="purple")
plt.title("Phase  Shift from the Drive and Deflection Frequencies")
plt.xlabel("Time (s)")
plt.ylabel("Phase Shift")
print(f"Time Lag (Delta t): {delta_t:.4f} s")


In [ ]:


# First curve_fit for driven_motion
fitted, _ = curve_fit(driven_motion, t_fake[0], totaldeflectionangle[0], p0=[0.1, 0.1, 0.1, 0.1, 0.1])
M0, phi, delta, J, omega0 = fitted
print("M0 = ", M0, "phi = ", phi, "C = ", C, "delta = ", delta, "alpha = ", alpha)

J = 0.132  
omega0 = 0.231

def equation_of_motion(t, M0, phi, C, delta, alpha):
    m = M0
    n = J * np.sqrt((omega0**2 - omega**2)**2 + (2 * delta * omega)**2)
    A = m / n
    return A * np.sin(omega * t + phi) + C * np.exp(-delta * t) * np.cos(omegad * t - alpha)

# Calculating angle
total_deflection = totaldeflectionangle[0]
driven_deflection = drivendeflection[0]
angle = total_deflection - driven_deflection

# Curve_fit for equation_of_motion
fitted, _ = curve_fit(equation_of_motion, t_fake[0], angle, p0=[0.1, 0.1, 1, 0.1, 0.1])
M0, phi, C, delta, alpha = fitted

# Calculating residual and determining fit quality
R2 = RR2(angle, equation_of_motion(t_fake[0], M0, phi, C, delta, alpha))
print("Sinusoidal Fit: ", R2)

# Resonance calculations
omegar = resonance_omega(omega0, delta)
q = quality_factor(omega0, delta)
print(f"Parameters for V: omegar = {omegar:.3f} and q = {q:.3f}")
print(f"Parameters for V: M0 = {M0:.3f}, phi = {phi:.3f}, C = {C:.3f}, delta = {delta:.3f}, alpha = {alpha:.3f}, omega0 = {omega0:.3f}, J = {J:.3f}")
final = equation_of_motion(t_fake[0], M0, phi, C, delta, alpha)
# Plot
plt.plot(t_fake[0], angle, label="Deflection Angle", color="blue")
plt.plot(t_fake[0], equation_of_motion(t_fake[0], M0, phi, C, delta, alpha), color="red")
plt.plot(t_common, angle_fit, color="red")
plt.plot(t_common, totaldeflectionangle[i], label="Driven Deflection", color="purple", zorder=1)
plt.plot(t_common, drivendeflection[i], label="Damping Deflection", color="green", zorder=1)
plt.plot(t_common, deflection_fit, color="red", label="Fits", zorder=2)
plt.title("Best Fit for Total Deflection")
plt.xlabel("Time (s)")
plt.ylabel("Deflection Angle (rad)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from scipy.optimize import curve_fit

M0 = -0.002
phi = 1.868
C = - 0.276
delta = 0.005
alpha = -6.674
omega0 = 0.231
J = 0.132
D = omega0**2 * J 
omegar = resonance_omega(omega0, delta)
q = quality_factor(omega0, delta)
Ar = resonance_amplitude(M0,omega0, J, D, delta)
print(f"Parameters for V: omegar = {omegar:.3f} and q = {q}, Ar = {Ar:.3f}")
A = M0 / (J * np.sqrt((omega0**2 - omega**2)**2 + (2* delta * omega)**2))
x_val = np.abs(omega/omega0)
y_val = A/Ar

plt.plot(omega/omega0, A/Ar, label="Resonance Curve", color="blue")
plt.title("Resonance Curve")
plt.xlabel("Frequency Ratio")
plt.ylabel("Amplitude Ratio")
plt.legend()
plt.grid(True)
plt.show()

try:
    fit,_ = curve_fit(cosinusoidal_model, x_val, y_val, p0=[1, 0.1, 0.1, 0.1], maxfev=5000)
    A, alpha, phi, C = fit
    amp_fit = cosinusoidal_model(x_val, A, alpha, phi, C)
    print(f"Parameters for V: A = {A:.3f}, phi = {phi:.3f} and C = {C:.3f}")
except RuntimeError as e:
    print(f"Curve fitting failed: {e}")
plt.plot(x_val,amp_fit, label="Fit", color="orange")
plt.plot(x_val, y_val, label="Resonance Curve", color="blue")
plt.title("Resonance Curve")
plt.xlabel("Frequency Ratio (ω/ω₀)")
plt.ylabel("Amplitude Ratio(A(ω)/A((ω)")
plt.legend()
plt.grid(True)
plt.show()
R2 = RR2(amp_fit, y_val)
print(f"Total r2 score: {R2}")

In [ ]:

def phase_shift_model(omega, omega0, delta):
    condition = omega < omega0
    p = 2 * omega * delta
    r1 = omega0**2 - omega**2
    r2 = omega**2 - omega0**2
    return np.where(condition, -np.arctan(p / r1), -np.pi + np.arctan(p / r2))


phaseshift = phase_shift_model(omega, omega0, delta)


plt.plot(omega / omega0, phaseshift, label="Phase Shift", color="blue")
plt.title("Phase Shift vs. Frequency")
plt.xlabel("Frequency Ratio (ω/ω₀)")
plt.ylabel("Phase Shift")
plt.legend()
plt.grid(True)
plt.show()


plt.plot(t_fake[0], phaseshift, label="Phase Shift", color="blue")
plt.title("Phase Shift vs. Time")
plt.xlabel("Time (s)")
plt.ylabel("Phase Shift")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
shifted_angle= np.interp(t_fake[0], t_fake[0] - phaseshift, final)
plt.plot(t_fake[0], final, label="Deflection Angle", color="blue")
plt.plot(t_fake[0], final + phaseshift, label="Shifted Deflection", color="purple")
plt.plot(t_common, angle_fit1[i], label="Driven Deflection",color="green")
plt.plot(t_common, deflection_fit1[i], label="Damping Deflection",color="red")
plt.title("Shifted Deflection by Computed Phase Shift") 
plt.xlabel("Time (s)")
plt.ylabel("Deflection Angle (rad)")
plt.legend()
plt.grid(True)
plt.show()